In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import re 
from ast import literal_eval

import matplotlib.pyplot as plt
import seaborn as sns

In [14]:
xcf_tb_data=pd.read_csv('~/work/food_flavor_2021/three_origins_ingredients_analysis/june_datasets/(14.09)xcf_tb_flavordb.csv',index_col=0,converters={'flavordb_ingrs':eval})
ars_tb_data=pd.read_csv('~/work/food_flavor_2021/three_origins_ingredients_analysis/june_datasets/(23.09)ars_tb_flavordb.csv',index_col=0,converters={'flavordb_ingr':eval})
kb_tb_data=pd.read_csv('~/work/food_flavor_2021/three_origins_ingredients_analysis/june_datasets/(09.07)kb_tb_flavordb.csv',index_col=0,converters={'flavordb_ingr':eval})

In [15]:
xcf_tb_data.rename(columns={'flavordb_ingrs':'flavordb_ingr'},inplace=True)

In [16]:
xcf_tb_data=xcf_tb_data[['class','recipe_id','flavordb_ingr']]

In [17]:
xcf_tb_data.head()

,class,recipe_id,flavordb_ingr
0,1,100000175,"[rice, salt, cabbage, oats, pepper]"
1,1,100000387,"[salt, broccoli, cooking_oil, pork]"
2,1,100000424,"[tomato, soybean_sauce, garlic, salt, cabbage,..."
3,1,100000444,"[shiitake, garlic, salt, cabbage, soybean_sauc..."
4,1,100000562,"[durian, water]"


In [18]:
ars_tb_data.head()

,class,recipe_id,flavordb_ingr
0,0,152,"[butter, buttermilk, cooking_oil, egg, flour, ..."
1,0,198,"[biscuit, butter, codfish, lemon, onion, parsl..."
2,0,313,"[apple, butter, capsicum, cheddar_cheese, egg,..."
3,0,382,"[beef, carrot, celery, garlic, kale, kidney_be..."
4,0,596,"[biscuit, chocolate, cream_cheese, egg, sugar,..."


In [19]:
kb_tb_data.head()

,class,recipe_id,flavordb_ingr
0,0,731,"[asparagus, butter, capsicum, chive, cooking_o..."
1,0,964,"[cocoa_powder, cream, egg, red_wine, sugar]"
2,0,1182,"[butter, egg, flour, milk, salt, spelt, sugar]"
3,0,1603,"[chive, onion, parsley, rosemary, water]"
4,0,1680,"[butter, egg, gelatin, lemon, milk, pudding, s..."


In [9]:
# drop the num.flavordb_ingrs
def calculate_frequency(data,column):
    #data['flvaordb_ingrs']=data['flvaordb_ingrs'].apply(list)
    data=data.explode(column)
    #data=pd.merge(data,flavordb_ingr,left_on='flvaordb_ingrs',right_on='entity_alias_readable')
    #print(data.shape)
    
    # delete duplicated ingredients in each recipe
    data=pd.DataFrame(data.groupby('recipe_id')[column].apply(set)).reset_index()
    data[column]=data[column].apply(list)
    data=data.explode(column)
    
    # calculate ingredient frequency
    ingr_fre=pd.DataFrame(data[column].value_counts()).reset_index()
    ingr_fre.columns=['ingr','ingr_fre']
    
    # find the ingredient pair
    list_data=pd.DataFrame(data.groupby('recipe_id')[column].apply(list)).reset_index()
    print('Number of recipes: ',list_data.shape)
    list_data[column]=list_data[column].apply(lambda x:sorted(x))
    list_data['pairs']=list_data[column].apply(lambda x:[(a,b) for idx,a in enumerate(x) for b in x[idx+1:]])
    pair_data=list_data.drop(column,axis=1)
    
    # calculate ingredient pair frequency
    pair_data=pair_data.explode('pairs')
    pair_fre=pd.DataFrame(pair_data['pairs'].value_counts()).reset_index()
    pair_fre.columns=['pairs','pair_frequency']
    
    return ingr_fre,pair_data,pair_fre

In [23]:
xcf_ingr_fre,xcf_pair_data,xcf_pair_fre=calculate_frequency(xcf_tb_data,'flavordb_ingr')

Number of recipes:  (2366, 2)


In [24]:
ars_ingr_fre,ars_pair_data,ars_pair_fre=calculate_frequency(ars_tb_data,'flavordb_ingr')

Number of recipes:  (2133, 2)


In [25]:
kb_ingr_fre,kb_pair_data,kb_pair_fre=calculate_frequency(kb_tb_data,'flavordb_ingr')

Number of recipes:  (1864, 2)


In [26]:
def cal_PMI(ingr_fre,pair_fre,N):
    pair_fre[['ingr1','ingr2']]=pd.DataFrame(pair_fre['pairs'].tolist(),index=pair_fre.index)
    temp=pd.merge(pair_fre,ingr_fre,left_on='ingr1',right_on='ingr').sort_values('pair_frequency',ascending=False)
    temp.drop('ingr',axis=1,inplace=True)
    temp.rename(columns={'ingr_fre':'ingr1_fre'},inplace=True)
    
    temp=pd.merge(temp,ingr_fre,left_on='ingr2',right_on='ingr').sort_values('pair_frequency',ascending=False)
    temp.drop('ingr',axis=1,inplace=True)
    temp.rename(columns={'ingr_fre':'ingr2_fre'},inplace=True)

    PMI_data=temp.copy()
    PMI_data['pair_prevelence']=PMI_data['pair_frequency']/N
    PMI_data['ingr1_prevelence']=PMI_data['ingr1_fre']/N
    PMI_data['ingr2_prevelence']=PMI_data['ingr2_fre']/N
    
    PMI_data.reset_index(drop=True,inplace=True)
    PMI_data['pmi']=np.log(PMI_data['pair_prevelence']/(PMI_data['ingr1_prevelence']*PMI_data['ingr2_prevelence']))
    
    return PMI_data

In [27]:
xcf_PMI_data=cal_PMI(xcf_ingr_fre,xcf_pair_fre,2366)
ars_PMI_data=cal_PMI(ars_ingr_fre,ars_pair_fre,2133)
kb_PMI_data=cal_PMI(kb_ingr_fre,kb_pair_fre,1864)

In [33]:
def cal_NPMI(pmi_data):
    pmi_data['npmi']=pmi_data['pmi']/-(np.log(pmi_data['pair_prevelence']))
    return pmi_data

In [34]:
xcf_PMI_data=cal_NPMI(xcf_PMI_data)
ars_PMI_data=cal_NPMI(ars_PMI_data)
kb_PMI_data=cal_NPMI(kb_PMI_data)

In [39]:
xcf_PMI_data.sort_values('npmi',ascending=False,inplace=True)
ars_PMI_data.sort_values('npmi',ascending=False,inplace=True)
kb_PMI_data.sort_values('npmi',ascending=False,inplace=True)

In [40]:
xcf_PMI_data.head()

,pairs,pair_frequency,ingr1,ingr2,ingr1_fre,ingr2_fre,pair_prevelence,ingr1_prevelence,ingr2_prevelence,pmi,npmi
383,"(mace, tamarind)",18,mace,tamarind,18,18,0.007608,0.007608,0.007608,4.878584,1.0
442,"(cardamom, mace)",18,cardamom,mace,18,18,0.007608,0.007608,0.007608,4.878584,1.0
381,"(tamarind, turmeric)",18,tamarind,turmeric,18,18,0.007608,0.007608,0.007608,4.878584,1.0
382,"(fenugreek, turmeric)",18,fenugreek,turmeric,18,18,0.007608,0.007608,0.007608,4.878584,1.0
388,"(saffron, turmeric)",18,saffron,turmeric,18,18,0.007608,0.007608,0.007608,4.878584,1.0


In [41]:
ars_PMI_data.head()

,pairs,pair_frequency,ingr1,ingr2,ingr1_fre,ingr2_fre,pair_prevelence,ingr1_prevelence,ingr2_prevelence,pmi,npmi
671,"(fenugreek, tamarind)",22,fenugreek,tamarind,22,22,0.010314,0.010314,0.010314,4.574242,1.000000
690,"(fenugreek, saffron)",22,fenugreek,saffron,22,24,0.010314,0.010314,0.011252,4.487231,0.980978
698,"(saffron, tamarind)",22,saffron,tamarind,24,22,0.010314,0.011252,0.010314,4.487231,0.980978
664,"(mace, tamarind)",22,mace,tamarind,25,22,0.010314,0.011721,0.010314,4.446409,0.972054
699,"(fenugreek, mace)",22,fenugreek,mace,22,25,0.010314,0.010314,0.011721,4.446409,0.972054


In [42]:
kb_PMI_data.head()

,pairs,pair_frequency,ingr1,ingr2,ingr1_fre,ingr2_fre,pair_prevelence,ingr1_prevelence,ingr2_prevelence,pmi,npmi
6369,"(dandelion, sorrel)",1,dandelion,sorrel,1,1,0.000536,0.000536,0.000536,7.530480,1.000000
6142,"(cassava, yam)",1,cassava,yam,1,1,0.000536,0.000536,0.000536,7.530480,1.000000
6227,"(passionfruit, soft_drink)",1,passionfruit,soft_drink,1,1,0.000536,0.000536,0.000536,7.530480,1.000000
2680,"(chicory, endive)",6,chicory,endive,6,6,0.003219,0.003219,0.003219,5.738721,1.000000
469,"(fenugreek, mace)",44,fenugreek,mace,45,44,0.023605,0.024142,0.023605,3.723818,0.994001


In [ ]:
# set threshold: ingredient_count >=10, pair_count>=5

In [43]:
filtered_xcf_PMI_data=xcf_PMI_data[(xcf_PMI_data['ingr1_fre']>=10)&(xcf_PMI_data['ingr2_fre']>=10)&(xcf_PMI_data['pair_frequency']>=5)]
filtered_xcf_PMI_data.shape

(1280, 11)

In [44]:
filtered_ars_PMI_data=ars_PMI_data[(ars_PMI_data['ingr1_fre']>=10)&(ars_PMI_data['ingr2_fre']>=10)&(ars_PMI_data['pair_frequency']>=5)]
filtered_ars_PMI_data.shape

(2341, 11)

In [45]:
filtered_kb_PMI_data=kb_PMI_data[(kb_PMI_data['ingr1_fre']>=10)&(kb_PMI_data['ingr2_fre']>=10)&(kb_PMI_data['pair_frequency']>=5)]
filtered_kb_PMI_data.shape

(3049, 11)

In [46]:
print('After filter, the number of ingredient pairs in \nxiachufang:{}-->{},\nallrecipes:{}-->{},\nkochbar:{}-->{}'
      .format(xcf_PMI_data.shape[0],filtered_xcf_PMI_data.shape[0],
              ars_PMI_data.shape[0],filtered_ars_PMI_data.shape[0],
              kb_PMI_data.shape[0],filtered_kb_PMI_data.shape[0]))

After filter, the number of ingredient pairs in 
xiachufang:4856-->1280,
allrecipes:8169-->2341,
kochbar:9758-->3049


In [47]:
filtered_xcf_PMI_data.head()

,pairs,pair_frequency,ingr1,ingr2,ingr1_fre,ingr2_fre,pair_prevelence,ingr1_prevelence,ingr2_prevelence,pmi,npmi
383,"(mace, tamarind)",18,mace,tamarind,18,18,0.007608,0.007608,0.007608,4.878584,1.0
442,"(cardamom, mace)",18,cardamom,mace,18,18,0.007608,0.007608,0.007608,4.878584,1.0
381,"(tamarind, turmeric)",18,tamarind,turmeric,18,18,0.007608,0.007608,0.007608,4.878584,1.0
382,"(fenugreek, turmeric)",18,fenugreek,turmeric,18,18,0.007608,0.007608,0.007608,4.878584,1.0
388,"(saffron, turmeric)",18,saffron,turmeric,18,18,0.007608,0.007608,0.007608,4.878584,1.0


In [48]:
filtered_ars_PMI_data.head()

,pairs,pair_frequency,ingr1,ingr2,ingr1_fre,ingr2_fre,pair_prevelence,ingr1_prevelence,ingr2_prevelence,pmi,npmi
671,"(fenugreek, tamarind)",22,fenugreek,tamarind,22,22,0.010314,0.010314,0.010314,4.574242,1.000000
690,"(fenugreek, saffron)",22,fenugreek,saffron,22,24,0.010314,0.010314,0.011252,4.487231,0.980978
698,"(saffron, tamarind)",22,saffron,tamarind,24,22,0.010314,0.011252,0.010314,4.487231,0.980978
664,"(mace, tamarind)",22,mace,tamarind,25,22,0.010314,0.011721,0.010314,4.446409,0.972054
699,"(fenugreek, mace)",22,fenugreek,mace,22,25,0.010314,0.010314,0.011721,4.446409,0.972054


In [49]:
filtered_kb_PMI_data.head()

,pairs,pair_frequency,ingr1,ingr2,ingr1_fre,ingr2_fre,pair_prevelence,ingr1_prevelence,ingr2_prevelence,pmi,npmi
469,"(fenugreek, mace)",44,fenugreek,mace,45,44,0.023605,0.024142,0.023605,3.723818,0.994001
535,"(mace, tamarind)",43,mace,tamarind,44,43,0.023069,0.023605,0.023069,3.746290,0.993901
498,"(fenugreek, tamarind)",43,fenugreek,tamarind,45,43,0.023069,0.024142,0.023069,3.723818,0.987939
497,"(saffron, tamarind)",43,saffron,tamarind,49,43,0.023069,0.026288,0.023069,3.638660,0.965346
522,"(mace, saffron)",43,mace,saffron,44,49,0.023069,0.023605,0.026288,3.615670,0.959247


In [ ]:
# keep the filter pmi data as pmi data

In [50]:
xcf_PMI_data=filtered_xcf_PMI_data.copy()
ars_PMI_data=filtered_ars_PMI_data.copy()
kb_PMI_data=filtered_kb_PMI_data.copy()

In [51]:
xcf_PMI_data.head()

,pairs,pair_frequency,ingr1,ingr2,ingr1_fre,ingr2_fre,pair_prevelence,ingr1_prevelence,ingr2_prevelence,pmi,npmi
383,"(mace, tamarind)",18,mace,tamarind,18,18,0.007608,0.007608,0.007608,4.878584,1.0
442,"(cardamom, mace)",18,cardamom,mace,18,18,0.007608,0.007608,0.007608,4.878584,1.0
381,"(tamarind, turmeric)",18,tamarind,turmeric,18,18,0.007608,0.007608,0.007608,4.878584,1.0
382,"(fenugreek, turmeric)",18,fenugreek,turmeric,18,18,0.007608,0.007608,0.007608,4.878584,1.0
388,"(saffron, turmeric)",18,saffron,turmeric,18,18,0.007608,0.007608,0.007608,4.878584,1.0


In [ ]:
# ingredient prevelence

In [58]:
xcf_ingr_fre['prevelence']=xcf_ingr_fre['ingr_fre']/2366
ars_ingr_fre['prevelence']=ars_ingr_fre['ingr_fre']/2133
kb_ingr_fre['prevelence']=kb_ingr_fre['ingr_fre']/1864

In [59]:
xcf_ingr_fre.head()

,ingr,ingr_fre,prevelence
0,sugar,1199,0.506762
1,salt,961,0.406171
2,egg,787,0.332629
3,flour,727,0.307270
4,water,499,0.210904


## form the ingr-ingr edge dataset

In [54]:
flavordb_info=pd.read_csv('~/work/food_flavor_2021/flavordb_dataset/(18.08)processed_flavordb_info.csv',index_col=0)

In [55]:
flavordb_info.head()

,entity_id,entity_alias_readable,category_readable,comb_cate
0,0,egg,animal product,Animal Product
1,1,bakery_products,bakery,Bakery
2,2,bread,bakery,Bakery
3,3,rye_bread,bakery,Bakery
4,4,wheaten_bread,bakery,Bakery


In [60]:
def form_network_dataset(npmi_data,flavordb_info,ingr_fre):
    # threshold = 0.10
    filtered_npmi_data=npmi_data[npmi_data['npmi']>=0.10]
    filtered_npmi_data=filtered_npmi_data[['ingr1','ingr2','npmi']]
    filtered_npmi_data.reset_index(drop=True,inplace=True)
    # get the edge data
    ingr_edge=filtered_npmi_data.copy()
    ingr_edge['edge_type']='ingr-ingr'
    print('There are {} edge'.format(ingr_edge.shape[0]))
    # extract node data from edge data
    nodelists=list(set(ingr_edge['ingr1'].values.tolist())|(set(ingr_edge['ingr2'].values.tolist())))
    print('There are {} nodes'.format(len(nodelists)))
    ingr_node=pd.DataFrame((pd.Series(nodelists)))
    ingr_node.rename(columns={0:'ingr'},inplace=True)
    # add attributes(cate, prevelence) to the node
    ingr_node=pd.merge(ingr_node,flavordb_info,left_on='ingr',right_on='entity_alias_readable')
    ingr_node=pd.merge(ingr_node,ingr_fre,left_on='ingr',right_on='ingr')
    ingr_node=ingr_node[['ingr','comb_cate','prevelence']]
    return ingr_edge,ingr_node

In [61]:
xcf_ingr_edge,xcf_ingr_node=form_network_dataset(xcf_PMI_data,flavordb_info,xcf_ingr_fre)

There are 751 edge
There are 130 nodes


In [65]:
ars_ingr_edge,ars_ingr_node=form_network_dataset(ars_PMI_data,flavordb_info,ars_ingr_fre)

There are 1144 edge
There are 143 nodes


In [66]:
kb_ingr_edge,kb_ingr_node=form_network_dataset(kb_PMI_data,flavordb_info,kb_ingr_fre)

There are 1527 edge
There are 155 nodes


In [105]:
xcf_ingr_edge.to_csv('datasets/(06.10)xcf_ingr_edge.pkl')
xcf_ingr_node.to_csv('datasets/(06.10)xcf_ingr_node.pkl')

ars_ingr_edge.to_csv('datasets/(06.10)ars_ingr_edge.pkl')
ars_ingr_node.to_csv('datasets/(06.10)ars_ingr_node.pkl')

kb_ingr_edge.to_csv('datasets/(06.10)kb_ingr_edge.pkl')
kb_ingr_node.to_csv('datasets/(06.10)kb_ingr_node.pkl')

## form the ingr_comp edge dataset

In [79]:
ingr_info=pd.read_csv('~/work/food_flavor_2021/flavordb_dataset/(19.08)propossed_ingr_comp.csv',index_col=0)
ingr_info.shape

(60263, 5)

In [80]:
ingr_info.head()

,ingredient_id,ingredient_name,compound_id,compound_name,CAS_number
0,0,egg,182,Dimethyl disulfide,624-92-0
1,0,egg,1095,L-histidine,71-00-1@26062-48-6
2,0,egg,45,Isobutyraldehyde,78-84-2
3,0,egg,127,Styrene,9003-53-6@100-42-5@98444-30-5@68441-35-0@12770...
4,0,egg,510,1-Undecanol,30207-98-8@143819-62-9@112-42-5


In [81]:
xcf_ingr_node['node_type']='ingr'
ars_ingr_node['node_type']='ingr'
kb_ingr_node['node_type']='ingr'

In [82]:
xcf_ingr_node.rename(columns={'ingr':'name'},inplace=True)
ars_ingr_node.rename(columns={'ingr':'name'},inplace=True)
kb_ingr_node.rename(columns={'ingr':'name'},inplace=True)

In [83]:
xcf_ingr_node.head()

,name,comb_cate,prevelence,node_type
0,mallard_duck,Animal Product,0.004227,ingr
1,pak_choy,Vegetable,0.004227,ingr
2,welsh_onion,Vegetable,0.118343,ingr
3,sausage,Dish,0.006340,ingr
4,cake,Dish,0.004227,ingr


In [84]:
xcf_ingrcomp_edge=pd.merge(xcf_ingr_node,ingr_info,left_on='name',right_on='ingredient_name')
ars_ingrcomp_edge=pd.merge(ars_ingr_node,ingr_info,left_on='name',right_on='ingredient_name')
kb_ingrcomp_edge=pd.merge(kb_ingr_node,ingr_info,left_on='name',right_on='ingredient_name')

In [86]:
xcf_ingrcomp_edge=xcf_ingrcomp_edge[['name','compound_name']]
ars_ingrcomp_edge=ars_ingrcomp_edge[['name','compound_name']]
kb_ingrcomp_edge=kb_ingrcomp_edge[['name','compound_name']]

In [88]:
xcf_ingrcomp_edge['edge_type']='ingr-comp'
ars_ingrcomp_edge['edge_type']='ingr-comp'
kb_ingrcomp_edge['edge_type']='ingr-comp'

In [89]:
xcf_ingrcomp_edge.head()

,name,compound_name,edge_type
0,mallard_duck,betaine,ingr-comp
1,mallard_duck,thiamine,ingr-comp
2,mallard_duck,CID 644104,ingr-comp
3,pak_choy,Geraniol,ingr-comp
4,pak_choy,isoliquiritigenin,ingr-comp


In [90]:
print(xcf_ingrcomp_edge.shape)
print(ars_ingrcomp_edge.shape)
print(kb_ingrcomp_edge.shape)

(15978, 3)
(17239, 3)
(19039, 3)


In [91]:
xcf_comp_node_list=list(set(xcf_ingrcomp_edge['compound_name'].values.tolist()))
ars_comp_node_list=list(set(ars_ingrcomp_edge['compound_name'].values.tolist()))
kb_comp_node_list=list(set(kb_ingrcomp_edge['compound_name'].values.tolist()))

In [92]:
xcf_comp_node=pd.DataFrame(xcf_comp_node_list,columns=['name'])
ars_comp_node=pd.DataFrame(ars_comp_node_list,columns=['name'])
kb_comp_node=pd.DataFrame(kb_comp_node_list,columns=['name'])

In [93]:
xcf_comp_node['node_type']='comp'
ars_comp_node['node_type']='comp'
kb_comp_node['node_type']='comp'

xcf_comp_node['comb_cate']='Chem'
ars_comp_node['comb_cate']='Chem'
kb_comp_node['comb_cate']='Chem'

In [94]:
xcf_ingrcomp_node=pd.concat([xcf_ingr_node,xcf_comp_node],axis=0).reset_index(drop=True)
ars_ingrcomp_node=pd.concat([ars_ingr_node,ars_comp_node],axis=0).reset_index(drop=True)
kb_ingrcomp_node=pd.concat([kb_ingr_node,kb_comp_node],axis=0).reset_index(drop=True)

In [95]:
xcf_ingrcomp_edge.rename(columns={'name':'ingr'},inplace=True)
ars_ingrcomp_edge.rename(columns={'name':'ingr'},inplace=True)
kb_ingrcomp_edge.rename(columns={'name':'ingr'},inplace=True)

In [ ]:
# form the full node and edge data(including ingr_ingr and ingr_comp)

In [97]:
xcf_ingr_edge.rename(columns={'ingr1':'id_1','ingr2':'id_2'},inplace=True)
ars_ingr_edge.rename(columns={'ingr1':'id_1','ingr2':'id_2'},inplace=True)
kb_ingr_edge.rename(columns={'ingr1':'id_1','ingr2':'id_2'},inplace=True)

In [96]:
xcf_ingrcomp_edge.rename(columns={'ingr':'id_1','compound_name':'id_2'},inplace=True)
ars_ingrcomp_edge.rename(columns={'ingr':'id_1','compound_name':'id_2'},inplace=True)
kb_ingrcomp_edge.rename(columns={'ingr':'id_1','compound_name':'id_2'},inplace=True)

In [98]:
xcf_full_edge=pd.concat([xcf_ingr_edge,xcf_ingrcomp_edge],axis=0).reset_index(drop=True)
ars_full_edge=pd.concat([ars_ingr_edge,ars_ingrcomp_edge],axis=0).reset_index(drop=True)
kb_full_edge=pd.concat([kb_ingr_edge,kb_ingrcomp_edge],axis=0).reset_index(drop=True)

In [99]:
xcf_full_node=xcf_ingrcomp_node.copy()
ars_full_node=ars_ingrcomp_node.copy()
kb_full_node=kb_ingrcomp_node.copy()

In [100]:
print('There are {} nodes in Xiachufang, from them there are {} ingredients and {} compounds'.format(xcf_full_node.shape[0],xcf_full_node[xcf_full_node['node_type']=='ingr'].shape[0],xcf_full_node[xcf_full_node['node_type']=='comp'].shape[0]))
print('There are {} nodes in Allrecipes, from them there are {} ingredients and {} compounds'.format(ars_full_node.shape[0],ars_full_node[ars_full_node['node_type']=='ingr'].shape[0],ars_full_node[ars_full_node['node_type']=='comp'].shape[0]))
print('There are {} nodes in Kochbar, from them there are {} ingredients and {} compounds'.format(kb_full_node.shape[0],kb_full_node[kb_full_node['node_type']=='ingr'].shape[0],kb_full_node[kb_full_node['node_type']=='comp'].shape[0]))

There are 1454 nodes in Xiachufang, from them there are 130 ingredients and 1324 compounds
There are 1515 nodes in Allrecipes, from them there are 143 ingredients and 1372 compounds
There are 1549 nodes in Kochbar, from them there are 155 ingredients and 1394 compounds


In [101]:
print('There are {} edges in Xiachufang, from them there are {} ingr-ingr and {} ingr-comp'.format(xcf_full_edge.shape[0],xcf_full_edge[xcf_full_edge['edge_type']=='ingr-ingr'].shape[0],xcf_full_edge[xcf_full_edge['edge_type']=='ingr-comp'].shape[0]))
print('There are {} edges in Allrecipes, from them there are {} ingr-ingr and {} ingr-comp'.format(ars_full_edge.shape[0],ars_full_edge[ars_full_edge['edge_type']=='ingr-ingr'].shape[0],ars_full_edge[ars_full_edge['edge_type']=='ingr-comp'].shape[0]))
print('There are {} edges in Kochbar, from them there are {} ingr-ingr and {} ingr-comp'.format(kb_full_edge.shape[0],kb_full_edge[kb_full_edge['edge_type']=='ingr-ingr'].shape[0],kb_full_edge[kb_full_edge['edge_type']=='ingr-comp'].shape[0]))

There are 16729 edges in Xiachufang, from them there are 751 ingr-ingr and 15978 ingr-comp
There are 18383 edges in Allrecipes, from them there are 1144 ingr-ingr and 17239 ingr-comp
There are 20566 edges in Kochbar, from them there are 1527 ingr-ingr and 19039 ingr-comp


In [106]:
xcf_full_node.to_csv('datasets/(06.10)xcf_full_node.pkl')
ars_full_node.to_csv('datasets/(06.10)ars_full_node.pkl')
kb_full_node.to_csv('datasets/(06.10)kb_full_node.pkl')

xcf_full_edge.to_csv('datasets/(06.10)xcf_full_edge.pkl')
ars_full_edge.to_csv('datasets/(06.10)ars_full_edge.pkl')
kb_full_edge.to_csv('datasets/(06.10)kb_full_edge.pkl')